In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset (replace 'your_dataset.csv' with the actual filename)
df = pd.read_csv('Breas cancer.csv')

# Handle missing values
df = df.fillna(method='ffill')  # Forward fill missing values

# Normalize the data
scaler = StandardScaler()
df['id=842302'] = scaler.fit_transform(df['id=842302'].values.reshape(-1, 1))

# Split into training and test sets (e.g., 80% training, 20% test)
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]

# Optionally, save the processed data
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)


KeyError: 'id=842302'

In [5]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor


def create_lstm_model(num_units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=num_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

regressor = KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=32, verbose=0)

param_grid = {
    'num_units': [50, 100, 150],
    'dropout_rate': [0.2, 0.3, 0.4],
}

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, cv=3)
grid_search_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_search_result.best_score_, grid_search_result.best_params_))


random_search = RandomizedSearchCV(estimator=regressor, param_distributions=param_grid, n_iter=10, cv=3)
random_search_result = random_search.fit(X_train, y_train)
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))


ModuleNotFoundError: No module named 'keras'

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Normalize the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Reshape input data to be 3D [samples, timesteps, features]
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_val_reshaped = X_val_scaled.reshape(X_val_scaled.shape[0], 1, X_val_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_val_reshaped, y_val), verbose=1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Evaluate the model
train_loss = model.evaluate(X_train_reshaped, y_train, verbose=0)
val_loss = model.evaluate(X_val_reshaped, y_val, verbose=0)
test_loss = model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f'Root Mean Squared Error: {rmse:.2f}')


NameError: name 'X_train' is not defined

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-total-female-births.csv'
df = pd.read_csv(url, header=0, parse_dates=[0], index_col=0, squeeze=True)

# Convert the dataframe to a supervised learning format
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    df = pd.DataFrame(data)
    cols = []
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    agg = pd.concat(cols, axis=1)
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

# Normalize the dataset
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df.values.reshape(-1, 1))

# Convert the dataset into a supervised learning problem
data = series_to_supervised(data_scaled, n_in=3, n_out=1)

# Split the data into input (X) and output (y) variables
X, y = data[:, :-1], data[:, -1]

# Reshape input data to be 3D [samples, timesteps, features]
X_reshaped = X.reshape(X.shape[0], 1, X.shape[1])

# Define the LSTM model function
def create_lstm_model(num_units=50, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=num_units, input_shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Wrap the model using KerasRegressor
model = KerasRegressor(build_fn=create_lstm_model, verbose=0)

# Define the hyperparameters grid
param_grid = {
    'num_units': [50, 100],
    'dropout_rate': [0.2, 0.3],
    'learning_rate': [0.001, 0.01]
}

# Perform grid search with time series cross-validation
tscv = TimeSeriesSplit(n_splits=3)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error')
grid_search_result = grid_search.fit(X_reshaped, y)

# Print best parameters and score
print("Best: %f using %s" % (grid_search_result.best_score_, grid_search_result.best_params_))


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tensorflow scikit-learn matplotlib


Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

(x_train, y_train), (x_test, y_test)  = keras.datasets.cifar10.load_data()



# Assuming X_train, y_train, X_val, y_val, X_test, y_test are already defined

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Evaluate the model on the test set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Hyperparameter tuning can be performed using techniques like grid search or random search
# Grid search example
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def create_lstm_model(units=50, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = KerasRegressor(build_fn=create_lstm_model, verbose=0)
param_grid = {
    'units': [50, 100],
    'dropout_rate': [0.2, 0.3],
    'learning_rate': [0.001, 0.01]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_search_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_search_result.best_score_, grid_search_result.best_params_))


NameError: name 'keras' is not defined

In [4]:
from tensorflow import keras

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 36s 0us/step
